In [1]:
import pandas as pd
import numpy as np
import re

# recurrence relation
$$\vec{p}_n = \vec{p}_0 + n\vec{v}_0 + \sum_{i=1}^{n}i\vec{a}_0$$

$$\vec{p}_n = \vec{p}_0 + n\vec{v}_0 +\frac{n(n+1)}{2}\vec{a}_0$$

In [2]:
test_data='''p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>'''.splitlines()
real_data = open('input.txt').readlines()

In [3]:
parser = re.compile(r'(.=<([^,]+),([^,]+),([^,]+)>(, )?)(.=<([^,]+),([^,]+),([^,]+)>(, )?)(.=<([^,]+),([^,]+),([^,]+)>(, )?)')

In [4]:
def parse_lines(lines):
    p = []
    v = []
    a = []
    for line in lines:
        l = parser.match(line)
        if l:
            p.append(list(map(int,l.group(2, 3, 4))))
            v.append(list(map(int,l.group(7, 8, 9))))
            a.append(list(map(int,l.group(12, 13, 14))))
    return map(lambda x: np.array(x, dtype=np.int64), [p, v, a]) # , dtype=np.float64

In [5]:
def manhattan(v):
    return abs(v).sum(axis=1)

def norm2(v):
    return (v**2).sum(axis=1)

In [6]:
# p, v, a = parse_lines(real_data)
p0, v0, a0 = parse_lines(real_data)
# n = 100000
# for i in range(n):
#     v = v + a
#     p = p + v

In [7]:
# def p_i(n):
#     return p0 + n*v0 + (n*(n+1)*a)/2

# np.where(manhattan(p_i(n*1000000)) == manhattan(p_i(n*1000000)).min())

# Part 1 - doesn't acceleration just win out?

particles `[ 21, 159, 285, 442, 457]` have the equal lowest acceleration.

particle 457 starts nearest the origin and moves slowest at the beginning (all of these particles are have a component of their velocity in the same direction as their respective accelerations, i.e. none is currently slowing)

therefore the part 1 answer is 457

In [8]:
test_data2 = '''p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>'''.splitlines()

In [9]:
#p0, v0, a0 = parse_lines(test_data2)
p0, v0, a0 = parse_lines(real_data)

## separate dataframes for $\vec{p}$, $\vec{v}$ and $\vec{a}$

In [10]:
iters = 0
p, v, a = map(pd.DataFrame, [p0, v0, a0])
lastcount = len(p) + 1

In [11]:
while iters < 100:
    if iters % 10000 == 0:
        print(f'iteration {iters:8}')
    v = v + a
    p = p + v
    mask = ~p.duplicated(keep=False)
    v = v[mask].reset_index(drop=True)
    a = a[mask].reset_index(drop=True)
    p = p[mask].reset_index(drop=True)
    count = len(p)
    iters += 1
    if count != lastcount:
        print(f'iteration {iters:8}: count = {count}', flush=True)
        lastcount = count



iteration        0
iteration        1: count = 1000
iteration       10: count = 979
iteration       12: count = 973
iteration       13: count = 955
iteration       14: count = 932
iteration       15: count = 921
iteration       16: count = 906
iteration       17: count = 874
iteration       18: count = 858
iteration       19: count = 831
iteration       20: count = 821
iteration       21: count = 809
iteration       22: count = 795
iteration       23: count = 791
iteration       24: count = 771
iteration       25: count = 752
iteration       26: count = 723
iteration       27: count = 703
iteration       28: count = 669
iteration       29: count = 648
iteration       30: count = 634
iteration       31: count = 622
iteration       32: count = 617
iteration       33: count = 589
iteration       34: count = 570
iteration       35: count = 542
iteration       36: count = 522
iteration       37: count = 494
iteration       38: count = 481
iteration       39: count = 448


In [12]:
len(p)

448

## single dataframe for $\vec{p}$, $\vec{v}$ and $\vec{a}$

In [13]:
p0, v0, a0 = parse_lines(real_data)


sdf = pd.concat(map(pd.DataFrame, [p0, v0, a0]), axis=1, ignore_index=True)
V = ['v0', 'v1', 'v2']
A = ['a0', 'a1', 'a2']
P = ['p0', 'p1', 'p2']

sdf.columns = P + V + A


In [14]:
sdf.head()

,p0,p1,p2,v0,v1,v2,a0,a1,a2
0,-833,-499,-1391,84,17,61,-4,1,1
1,-168,3586,-2721,-61,-58,61,7,-13,8
2,364,223,1877,31,-11,-71,-5,0,-3
3,769,-854,-8705,-20,4,64,0,1,9
4,6985,-3666,3653,-112,99,-23,-4,0,-4


In [15]:
from IPython.display import display, HTML
iters = 0
lastcount = len(sdf)+1 # set higher than he length, so that the count will be displayed on the first iteration
verbose=False

In [16]:
#display(sdf)
while iters < 100:
    sdf.loc[:, V] += sdf.loc[:, A].rename(columns=dict(zip(A, V)))
    sdf.loc[:, P] += sdf.loc[:, V].rename(columns=dict(zip(V, P)))
    #display(sdf)
    mask = ~sdf.duplicated(subset=P, keep=False) # keep = False == don't include ANY dupes in output
    if verbose:
        if sum(~mask) != 0:
            display(HTML('Collision between particles:'))
            display(sdf[~mask])
    sdf = sdf[mask]
    count = len(sdf)
    iters += 1
    if count != lastcount:
        print(f'iteration {iters:8}: count = {count}', flush=True)
        lastcount = count


iteration        1: count = 1000
iteration       10: count = 979
iteration       12: count = 973
iteration       13: count = 955
iteration       14: count = 932
iteration       15: count = 921
iteration       16: count = 906
iteration       17: count = 874
iteration       18: count = 858
iteration       19: count = 831
iteration       20: count = 821
iteration       21: count = 809
iteration       22: count = 795
iteration       23: count = 791
iteration       24: count = 771
iteration       25: count = 752
iteration       26: count = 723
iteration       27: count = 703
iteration       28: count = 669
iteration       29: count = 648
iteration       30: count = 634
iteration       31: count = 622
iteration       32: count = 617
iteration       33: count = 589
iteration       34: count = 570
iteration       35: count = 542
iteration       36: count = 522
iteration       37: count = 494
iteration       38: count = 481
iteration       39: count = 448


two particles $P$ and $P'$ with parameters $\{\vec{p}, \vec{v}, \vec{a}\}$ and $\{\vec{p'}, \vec{v'}, \vec{a'}\}$ collide at time n, if and only if the following are equal:

$$\vec{p}_n = \vec{p}_0 + n\vec{v}_0 +\frac{n(n+1)}{2}\vec{a}_0$$

$$\vec{p'}_n = \vec{p'}_0 + n\vec{v'}_0 +\frac{n(n+1)}{2}\vec{a'}_0$$

i.e. if

$$\vec{p}_0 + n\vec{v}_0 +\frac{n(n+1)}{2}\vec{a}_0 = \vec{p'}_0 + n\vec{v'}_0 +\frac{n(n+1)}{2}\vec{a'}_0$$

$$(\vec{p}_0 - \vec{p'}_0) + n(\vec{v}_0 - \vec{v'}_0) + \frac{n(n+1)}{2}(\vec{a}_0 -\vec{a'}_0) = 0$$